In [ ]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import re
import xml.etree.ElementTree as ET
import xmltodict
import collections
import itertools
from scipy import stats
import statsmodels.api as sm 

import matplotlib.pyplot as plt
from ratelimit import limits, sleep_and_retry
# Import API key


In [ ]:
StateSelected = ['GA']

In [ ]:
zipcodes=pd.read_csv('free-zipcode-database-Primary.csv')
zipcodes.head()

In [ ]:
gazipcodes= zipcodes[zipcodes.State.isin(StateSelected)]

In [ ]:
gazipcodes=gazipcodes.head(10)
len(gazipcodes)

## Perform API Calls

In [ ]:

#conn = http.client.HTTPSConnection("search.onboard-apis.com") 

params = { 
    'accept': "application/json", 
    'apikey': "9d078487e223b1c4d54c3f3a3f628803", 
} 

In [ ]:
FullAddress=[]
Street=[]
CityStateZip=[]
Latitude=[]
Longitude=[]
pd.Series(FullAddress)
pd.Series(Latitude)
pd.Series(Longitude)
Data_set=pd.DataFrame({'FullAddress':pd.Series(FullAddress),'Street':pd.Series(Street),
                       'CityStateZip':pd.Series(CityStateZip),'Latitude':pd.Series(Latitude),'Longitude':pd.Series(Longitude)})
url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode=postalcode&page=1&pagesize=100"
for index, row in gazipcodes.iterrows():
    postalcode=row['Zipcode']
    url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode={postalcode}&page=1&pagesize=1000"
    res = requests.get(url,headers=params)
    data =json.loads(res.text)
    we=data
    try:
        d=pd.DataFrame(data['property'])
        d['FullAddress']=d['address'].apply(lambda a:a['oneLine'])
        d['Street']=d['address'].apply(lambda d:d['line1'])
        d['CityStateZip']=d['address'].apply(lambda d:d['line2'])
        d['Latitude']=d['location'].apply(lambda c:c['latitude'])
        d['Longitude']=d['location'].apply(lambda b:b['longitude'])
        datatoappend=d[['FullAddress','Street','CityStateZip','Latitude','Longitude']]
        Data_set=Data_set.append(datatoappend)
        
    
            
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    


In [ ]:
Data_set.head()

In [ ]:
Data_set['Zilliw format add']=Data_set['Street'].replace(" ", "+",regex=True)
Data_set['Zilliw format Zipciy']=Data_set['CityStateZip'].replace(" ", "+",regex=True)


In [ ]:

DataSet=Data_set.head(100)


In [ ]:
# C:\Users\cuong\Downloads\data_random3.csv
Data_set2 = pd.read_csv("C:/Users/cuong/Downloads/data_random3.csv")
Data_set2 = Data_set2.head(1000)
Data_set2

In [ ]:
#Next is to feed the addresess to zillow api
# OpenWeatherMap API Key
# X1-ZWz18cbenu9jij_85kuc ----- Cuong API
zid = 'X1-ZWz1go72alddzf_8469v'
#api_key = api_keys.api_key

In [ ]:
zpid=[]
zest=[]
addr =[]
city=[]
zipC=[]
type=[]
rooms=[]
sqft=[]
state=[]
lastsoldprice=[]
lastsolddate=[]
year_built=[]
c_value=[]
county=[]
lat =[]
lon =[]
Zindex =[]

for index, row in Data_set2.iterrows():
    add=row['Zilliw format add']
    statezip=row['Zilliw format Zipciy']
    params2=f'{add}&citystatezip={statezip}'
    url2 = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={zid}&address={params2}"

   
    response2 = requests.get(url2, params=params2)
 
    we=response2.content
   
    tree=ET.fromstring(we)
    results=tree.findall('./response/results/result')
    
    if len(results)!=0:
        re=results[0] 
        dic=xmltodict.parse(ET.tostring(re))
        try: 
            
             Zid=dic['result'].get('zpid',np.nan)
             zestimate=dic['result']['zestimate']['amount'].get('#text',np.nan)
             strt=dic['result']['address'].get('street',np.nan)
             zip=dic['result']['address'].get('zipcode',np.nan)
             cit=dic['result']['address'].get('city',np.nan)
             price=dic['result'].get('lastSoldPrice',np.nan)
             solddate=dic['result'].get('lastSoldDate',np.nan)
             room=dic['result'].get('bedrooms',np.nan)
             usecode=dic['result'].get('useCode',np.nan)
             sqftt=dic['result'].get('finishedSqFt',np.nan)
             yearbuilt=dic['result'].get('yearBuilt',np.nan)
             contyvalue=dic['result'].get('taxAssessment',np.nan)
             Zindx =dic['result'].get('localRealEstate',np.nan)
             year_built.append(yearbuilt)
             lati=dic['result']['address'].get('latitude',np.nan)
             longi=dic['result']['address'].get('longitude',np.nan)
             lon.append(longi)
             lat.append(lati)
             zest.append(zestimate)
             zpid.append(Zid)
             addr.append(strt)
             zipC.append(zip)
             city.append(cit)
             lastsoldprice.append(price)
             rooms.append(room)
             type.append(usecode)
             sqft.append(sqftt)
             c_value.append(contyvalue)
             lastsolddate.append(solddate)
             Zindex.append(Zindx)
             longi=[]
             lati=[]
             Zindx=[]
             solddate=[]
             zestimate=[]
             Zid=[]
             price=[]
             room=[]
             usecode=[]
             sqftt=[]
             cit=[]
             usecode=[]
             yearbuilt=[]
             contyvalue=[]
             strt=[]
             print(f'got it updating')
             
        except (KeyError, IndexError):
             print(skip)
        


In [ ]:
re

In [ ]:
d=pd.DataFrame({'Zillow Id':pd.Series(zpid),'Zestimate':pd.Series(zest),'Street':pd.Series(addr),\
                'City':pd.Series(city),'ZipCode':pd.Series(zipC),'lastSold':pd.Series(lastsoldprice),\
                'Bedrooms':pd.Series(rooms),'typeofhome':pd.Series(type),'Squareft':pd.Series(sqft),\
              'YearBuilt':pd.Series(year_built),'CountTax':pd.Series(c_value),\
              'Latitude':pd.Series(lat),'Longitude':pd.Series(lon),'LastSoldDate':pd.Series(lastsolddate),\
                'Z_index':pd.Series(Zindex)})
              

In [ ]:
df=d.drop_duplicates(['Zillow Id'])
df.head()

In [ ]:
dic

In [ ]:
df['LastSold']=df['lastSold'].apply(lambda l: l.get('#text', np.nan) if isinstance(l, dict) else np.nan)
df['Zindex']=df['Z_index'].apply(lambda l: l.get('region', np.nan) if isinstance(l, dict) else np.nan)
df['Z_index']=df['Zindex'].apply(lambda l: l.get('zindexValue', np.nan) if isinstance(l, dict) else np.nan)
df.head(10)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
selected=['Zestimate', 'ZipCode',\
           'Bedrooms', 'typeofhome', 'Squareft', 'YearBuilt', 'CountTax',\
           'LastSold','Z_index']

In [ ]:
dfreg=df[selected]


In [ ]:
dfreg['typeofhome'].value_counts()

In [ ]:
dfreg

In [ ]:
dselect=dfreg[['Zestimate', 'Bedrooms','YearBuilt','Squareft', 'CountTax', 'LastSold']]
#dselect['Z_index']=pd.to_numeric(dselect['Z_index'],errors='coerce',downcast='integer')
dselect=dselect.dropna(how='any')
#X,Y=dselect['Zestimate'],dselect['Bedrooms','YearBuilt']
dselect['YearBuilt']=pd.to_numeric(dselect['YearBuilt'])

X = dselect[['Bedrooms','Squareft','Zestimate','CountTax']]
Y = dselect['LastSold']

In [ ]:
X1 = sm.add_constant(X)
reg = sm.OLS(Y.astype(float), X1.astype(float)).fit()

reg.summary()

In [ ]:
plt.scatter(X,Y)
plt.ylabel('Zestimate')
plt.xlabel('Bedrooms')
plt.show()

In [ ]:
# df.to_csv("CuongPart3.csv")